In [3]:
import sys
import time
import torch
import torch.optim as optim
import pickle
import scipy as sp
from scipy import spatial
import numpy as np
from sklearn.cluster import KMeans
#from dge_fun import *
#from kmeans import lloyd



#device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(device)
torch.cuda.is_available()

# FORCE CPU for timing purpose
#device = torch.device("cpu")

#load cleaned data
path_dir = ""
save_path = ""
lossdata_path = ""



cuda


In [4]:
#--- FUNCTION DEFINITONS --> PUT INTO MODULE
#--- ? HOW TO PASS ON "device" TO MODULE ?

def class_sim_torch(X):
    Tpxy=torch.sum(torch.abs(X - X[:, None]),2)
    return Tpxy

def PDIST(X):
    #print(X.type())
    N=X.size()[0]
    dim=X.size()[1]
    numel=N*N*dim
    numel_max = 400000000
    #numel_max = 100000000
    nl=np.ceil(numel/numel_max).astype(int) # divide dim by this
    if nl ==1:
        Tpxy=torch.sum(torch.abs(X - X[:, None]),2)
    else:
        Tpxy=torch.zeros(N,N).to(device)
        Tpxy=Tpxy.type(X.type())
        mdim=np.floor(dim/nl).astype(int) # dim slices
        for i in range(nl+1):
            i1=i*mdim
            i2=np.minimum((i+1)*mdim,dim)
            if i1<i2:
                Xt=X[:,i1:i2]
                Tpxy.add_(torch.sum(torch.abs(Xt - Xt[:, None]),2))    
    return Tpxy

def NLSS( X, param,r_loc):

    if len(sys.argv)<3:
        r_loc=[]
        
    N = len(X)
    dim = X.shape[1]
    #--- compute distance & similarity
    XM2 = sp.spatial.distance.squareform(sp.spatial.distance.pdist(X,metric = 'cityblock'))/dim
    XSS0 = np.exp(-1/param*XM2)

    # if not NLM over whole sequence, set similarity beyond search radius to
    # zero and extend boundaries
    if r_loc:   
        t=np.arange(1,N)
        for i in range(1,N):
            t = np.concatenate((t, np.arange(1,N-i)), axis=None)
        t = np.transpose(t)
        T=spatial.distance.squareform(t) #%T=T(:,1:min(N1,N2));
        MASK=sp.exp(math.log(0.25)*T/r_loc)
        XSS=np.multiply(XSS0,MASK)
        XSS = np.transpose(XSS)
        XSS= np.concatenate((np.flipud(XSS[0:r_loc,:]),XSS,np.flipud(XSS[XSS.shape[0]-r_loc:XSS.shape[0],:])), axis=0)
        XSS = np.transpose(XSS)
    return XSS

def construct_SS_matrix(X,nnh):
# function [XX] = construct_SS_matrix(X,nnh)
# 
# stack nnh left-and right neighbor feature vectors of X using periodic
# boundary conditions.  
# size(X)  = N_frames x N_features 
# size(XX) = N_frames x N_features* 2*nnh 

    # periodic border effects
    XE=np.concatenate((  np.flipud(X[1:nnh+1,:]),X,(np.flipud(X[X.shape[0]-nnh-2:X.shape[0]-1,:]))), axis = 0)
    
    #--- construct neighborhood 
    a= np.arange(-nnh,0, dtype = int)
    b = np.arange(1,nnh+1, dtype = int)

    NH_ID = np.concatenate((a,b))# do not include self
    #NH_ID = np.concatenate((a,0, b))% include self (i.e., "center pixel")

    if not NH_ID.any():
        # nothing to do: working with pixel only, without neighbors
        XX=X
    else:
        # stack neighbors
        XX=np.array([]) 
    for inh in NH_ID:
        XTMP= np.roll(XE,inh,axis=0) 
        if not XX.size:  
            XX = XTMP[nnh:XTMP.shape[0]-nnh-1,:]
        else:
            XX=np.hstack((XX,XTMP[nnh:XTMP.shape[0]-nnh-1,:]))
    return XX

def NLSS_torch( X, param,r_loc):

    if len(sys.argv)<3:
        r_loc=[]
        
    N = X.size()[0]
    dim = X.size()[1]
    #--- compute distance & similarity
    XM2 = PDIST(X)/dim
    XSS0 = torch.exp(-1/param*XM2).double()
    # if not NLM over whole sequence, set similarity beyond search radius to
    # zero and extend boundaries
    if r_loc:   
        t1=torch.zeros(N,1).to(device)
        t1[:,0]=torch.arange(0,N)
        T=PDIST(t1)
        mparam=-1.3862943611198906/r_loc
        MASK=torch.exp(mparam*T).double()
        XSS=torch.mul(XSS0,MASK)
        XSS = XSS.transpose(1,0)
        
        np1=torch.flip(XSS[0:r_loc,:],(0,))        
        np2=torch.flip(XSS[XSS.size()[0]-r_loc:XSS.size()[0],:],(0,))
        XSS = torch.cat((np1, XSS, np2))
        XSS = XSS.transpose(1,0)
    return XSS

def PCA(X, k=2):
    # preprocess the data
    X_mean = torch.mean(X,0)
    X = X - X_mean.expand_as(X)
    # svd
    U,S,V = torch.svd(torch.t(X))
    return U[:,:k], torch.mm(X,U[:,:k])

def distx_fun(X):
    N = X.size()[0]
    dim=X.size()[1]

    Z = torch.mm(X,X.t())#.to(device)
    djj = torch.sqrt(torch.diag(Z))*torch.ones(1,N).double().t().to(device)
    Z = torch.div(1 - torch.div(torch.div(Z,djj.t()), djj) , dim)
    return Z
    
def simz_fun(Z,param):
    G = torch.exp(torch.mul(-1/param,Z)) 
    N=G.size()[1]
    z = torch.sum(G,0)
    G=torch.div(G,z.repeat(N,1));
    return G

def norm11(X):
    for i in range(0,X.size()[1]):
        X[:,i]=X[:,i]-X[:,i].min()
        mm=X[:,i].max()
        if mm is not 0:
            X[:,i]=X[:,i]/mm*2-1
    return X

def loss_fun(G,W):
    N=G.size()[0]
    # cross-entropy loss 
    w=W.contiguous().view(N*N,-1)
    g=G.contiguous().view(N*N,-1)
    L=-torch.sum(torch.mul(w,torch.log(g)))
    L=L+torch.sum(torch.mul(w,torch.log(w)))
    return L

def d_distx_fun(X):
    tt=X.size()
    N=tt[0]
    dim=tt[1]

    Z = torch.mm(X,X.t()) + 1e-16
    N = Z.size()[1];
    djj = torch.sqrt(torch.diag(Z))*torch.ones(1,N).double().t().to(device)
    dii= djj.t()
    t1 = torch.div(torch.div(1,dii), djj)
    t1 = t1 + torch.diag(torch.diag(t1)) # OK
    t2 = t1*torch.div(torch.div(Z,djj), djj) # OK

    X1=X
    X1.detach().clone()
    X1=X1.repeat((N,1,1))
    X2=X1
    X2.detach().clone()
    X2=X2.transpose(0,1)

    T1=t1.repeat((dim,1,1))
    T1=T1.transpose(0,1).transpose(1,2)
    T2=t2.repeat((dim,1,1))
    T2=T2.transpose(0,1).transpose(1,2)

    #dZ=X2*T1 - X1*T2
    dZ=torch.mul(X2,T1) - torch.mul(X1,T2)
    dZ=-dZ
    return dZ 
    
def d_simz_fun(Z,dZ,param):
    tt=dZ.size()
    N=tt[0]
    dim=tt[2]
    G = torch.exp(torch.mul(-1/param,Z))
    dG0=-1/param*G
    dG0=dG0.repeat((dim,1,1))
    dG0=dG0.transpose(0,1).transpose(1,2)
    #dG=dZ*dG0
    dG=torch.mul(dZ,dG0)

    sG=torch.sum(G,0).repeat((N,1)).repeat((dim,1,1)).transpose(0,1).transpose(1,2)
    sdG=torch.sum(dG,0).repeat((N,1,1))

    repG=G.repeat((dim,1,1)).transpose(0,1).transpose(1,2)
    dG=torch.div((dG*sG - repG*sdG) , (sG*sG))
    return dG

def d_loss_fun(G,dG,W):
    tt=dG.size()
    N=tt[0]
    dim=tt[2]
    WG=torch.div(W,G).repeat((dim,1,1)).transpose(0,1).transpose(1,2)
    dL=-torch.sum(dG*WG,0)
    return dL

def full_grad_fun(X,W,param):
    Z = distx_fun(X)
    dZ = d_distx_fun(X)
    G = simz_fun(Z,param)
    dG = d_simz_fun(Z,dZ,param)
    dL = d_loss_fun(G,dG,W)
    return dL, G

def sim_grad_descent(X,W,param,niter):
    eta0=1e-6
    N=X.size()[0]
    dim=X.size()[1]
    # step 1
    dL,G=full_grad_fun(X,W,param) # compute gradient
    x1=X.permute(1,0)
    g1=dL.permute(1,0)
    x1=x1.contiguous().view(-1,N*dim)
    g1=g1.contiguous().view(-1,N*dim)
    eta=eta0
    dataiter=X-eta*dL;

    # step 2-niter
    for it in range(niter-1):
        dL,G=full_grad_fun(dataiter,W,param) # compute gradient
        x2=x1
        g2=g1
        x2.detach().clone()
        g2.detach().clone()
        x1=dataiter.permute(1,0)
        g1=dL.permute(1,0)
        x1=x1.contiguous().view(-1,N*dim)
        g1=g1.contiguous().view(-1,N*dim)
        dx=x1-x2
        dg=g1-g2;
        eta=torch.mm(dg,dx.transpose(1,0)) / torch.mm(dg,dg.transpose(1,0))

        if eta != eta: # eta is nan. this actually means that the gradient is zero: STOP
            #print(g1)
            eta=1e-16
        
        dataiter=dataiter - eta*dL
    
    return dataiter

def sim_grad_descent_autograd(X,W,param,niter):
    eta0=1e-6
    N=X.size()[0]
    dim=X.size()[1]
    # step 1
    
    loss=loss_fun(simz_fun(distx_fun(X),param),W)
    loss.backward()
    dL=X.grad

    x1=X.permute(1,0)
    g1=dL.permute(1,0)
    x1=x1.contiguous().view(-1,N*dim)
    g1=g1.contiguous().view(-1,N*dim)
    eta=eta0
    X=X-eta*dL
    X=X.detach().clone()
    X.requires_grad=True

    # step 2-niter
    for it in range(niter-1):
        loss=loss_fun(simz_fun(distx_fun(X),param),W)
        loss.backward()
#        dL=X.grad.detach().clone()
        dL=X.grad
        x2=x1
        g2=g1
        x2.detach().clone()
        g2.detach().clone()
        x1=X.permute(1,0)
        g1=dL.permute(1,0)
        x1=x1.contiguous().view(-1,N*dim)
        g1=g1.contiguous().view(-1,N*dim)
        dx=x1-x2
        dg=g1-g2
        eta=torch.mm(dg,dx.transpose(1,0)) / torch.mm(dg,dg.transpose(1,0))

        if eta != eta: # eta is nan. this actually means that the gradient is zero: STOP
            #print(g1)
            eta=1e-16
        
        X=X - eta*dL
        X=X.detach().clone()
        X.requires_grad=True
    
    return X

def sim_grad_descent_biW(X,W1,W2,param,aa,niter):
    eta0=1e-6
    aa=min(max(aa,0),1)
    a1,a2=aa,1-aa
    N=X.size()[0]
    dim=X.size()[1]
    # step 1
    dL1,G1=full_grad_fun(X,W1,param) # compute gradient
    dL2,G2=full_grad_fun(X,W2,param) # compute gradient
    dL=a1*dL1+a2*dL2
    G=a1*G1+a2*G2
    x1=X.permute(1,0)
    g1=dL.permute(1,0)
    x1=x1.contiguous().view(-1,N*dim)
    g1=g1.contiguous().view(-1,N*dim)
    eta=eta0
    dataiter=X-eta*dL;

    # step 2-niter
    for it in range(niter-1):
        dL1,G1=full_grad_fun(dataiter,W1,param) # compute gradient
        dL2,G2=full_grad_fun(dataiter,W2,param) # compute gradient
        dL=a1*dL1+a2*dL2
        G=a1*G1+a2*G2
        x2=x1
        g2=g1
        x2.detach().clone()
        g2.detach().clone()
        x1=dataiter.permute(1,0)
        g1=dL.permute(1,0)
        x1=x1.contiguous().view(-1,N*dim)
        g1=g1.contiguous().view(-1,N*dim)
        dx=x1-x2
        dg=g1-g2;
        eta=torch.mm(dg,dx.transpose(1,0)) / torch.mm(dg,dg.transpose(1,0))

        if eta != eta: # eta is nan. this actually means that the gradient is zero: STOP
            #print(g1)
            eta=1e-16
        
        dataiter=dataiter - eta*dL
    
    return dataiter

def loss_fun_biW(G,W1,W2,aa):
    N=G.size()[0]
    aa=min(max(aa,0),1)
    a1,a2=aa,1-aa
    # cross-entropy loss 
    w1=W1.contiguous().view(N*N,-1)
    w2=W2.contiguous().view(N*N,-1)
    g=G.contiguous().view(N*N,-1)
    L=-a1*torch.sum(torch.mul(w1,torch.log(g)))-a2*torch.sum(torch.mul(w2,torch.log(g)))
    L=L+a1*torch.sum(torch.mul(w1,torch.log(w1)))+a2*torch.sum(torch.mul(w2,torch.log(w2)))
    return L

def sim_grad_descent_biW_autograd(X,W1,W2,param,aa,niter):
    eta0=1e-6
    N=X.size()[0]
    dim=X.size()[1]
    # step 1
    
    loss=loss_fun_biW(simz_fun(distx_fun(X),param),W1,W2,aa)
    loss.backward()
    dL=X.grad

    x1=X.permute(1,0)
    g1=dL.permute(1,0)
    x1=x1.contiguous().view(-1,N*dim)
    g1=g1.contiguous().view(-1,N*dim)
    eta=eta0
    X=X-eta*dL
    X=X.detach().clone()
    X.requires_grad=True

    # step 2-niter
    for it in range(niter-1):
        loss=loss_fun_biW(simz_fun(distx_fun(X),param),W1,W2,aa)
        loss.backward()
        dL=X.grad
        x2=x1
        g2=g1
        x2.detach().clone()
        g2.detach().clone()
        x1=X.permute(1,0)
        g1=dL.permute(1,0)
        x1=x1.contiguous().view(-1,N*dim)
        g1=g1.contiguous().view(-1,N*dim)
        dx=x1-x2
        dg=g1-g2
        eta=torch.mm(dg,dx.transpose(1,0)) / torch.mm(dg,dg.transpose(1,0))

        if eta != eta: # eta is nan. this actually means that the gradient is zero: STOP
            #print(g1)
            eta=1e-16
        
        X=X - eta*dL
        X=X.detach().clone()
        X.requires_grad=True
    
    return X



<>:151: SyntaxWarning: "is not" with a literal. Did you mean "!="?
<>:151: SyntaxWarning: "is not" with a literal. Did you mean "!="?
<ipython-input-4-c54c7a38bf28>:151: SyntaxWarning: "is not" with a literal. Did you mean "!="?
  if mm is not 0:


In [5]:
from scipy.io import loadmat
import pandas as pd
import glob, os
import h5py
from baseline_agg import mean_aggregation, get_dist, get_prediction_signal
from scipy.spatial.distance import cosine
from scipy.signal import argrelmax
from testing_utils import evaluation_measures
all_scores = {}
scores = evaluation_measures(win=5)

dataset = "/home/hwendt/jupyter_MD/ACMMM2019/CES/EDUB-Seg.h5"

path_dir = "/media/DATA/DATASET/EDUB-Seg/"


f = h5py.File(dataset, 'r') 
user = f['user_id']
day = f['day']
gt = f['boundary']

samples = pd.DataFrame(np.vstack((user, day)).T)
samples = samples.drop_duplicates().values
i=1
for u, d in samples:
    mask = (user[:]==u)*(day[:]==d)
    GT=gt[mask,]
    #np.savetxt(path_dir + "GT/h5_Subject" + str(u) + "_Set"+str(d+1)+".txt",GT)
    np.save(path_dir + "GT/seq_" + str(i),GT)
    i=i+1
    


def mean_aggregation(feat, N=10):
    C = [np.mean(feat[(i-N):i], axis=0) for
         i in range(N, len(feat)+1)]
    P = np.vstack(([np.mean(feat[:i], axis=0) for
                    i in range(1, N)], C))
    F = np.vstack((C, 
                   [np.mean(feat[(-N + i):], axis=0) for
                    i in range(1, N)]))
    return P, F


def boundary_prediction(embedded_seqFfromP, embedded_seqPfromF,
                        order=5, level=1):
    """Predicts the event boundaries at time t given
    the past and future visual context.
    Inputs:
        visual context given the past
        visual context given the future
        order=size of the window for the local maxima  
        level=hierarchy level for the event segmentation
    Output:
        prediction vector (1/0)
    """
    # at time t, we want the difference between context from the past at t-1
    # and context from the future at t+1:
    signal = np.array([cosine(a, b) for
                       (a, b) in zip(embedded_seqFfromP[:-2],
                                     embedded_seqPfromF[2:])])

    # First and last context predictions will be noisy, better to ignore:
    signal[0] = 0
    signal[-1] = 0

    local_max = argrelmax(signal, order=order)[0]
    th = np.mean(signal[local_max])
    if level != 1:
        th += (1-level)*np.std(signal[local_max])
    prediction = np.zeros(len(signal))
    prediction[local_max[signal[local_max] > th]] = 1

    # First and last frames are always event boundaries 
    # (context prediction goes from t=1 to t=N-1):
    return np.hstack((1, prediction, 1))
    


ModuleNotFoundError: No module named 'baseline_agg'

In [6]:
FPTH = ""
SPTH = FPTH + 'NL_SS_FINAL/self_loc_L1_'


SAVEDIR = ""

N_SEQ=20
SNME='EDUB-Seg'

# master parameter for similarity
par00=0.25 

# search radius and neighborhood for NLmean
r_loc=3
nnh=1
NLSstr= '_NH' + str(nnh) + '_R' + str(r_loc) + '_'+str(par00)

# parameters for initial embedding
param2=par00/0.8
param1=par00/50/2
d_emb = 15
niter_e=150
embstr = '_emb_dim' + str(d_emb)

# parameters for dynamic graph embedding
N_iter_G=10 # number of DGE iterations
niter_g=5  # embedding steps
aa=0.3     # proportionality factor for initial matrix W0
aa2=1e-3   # strengthen diagonal
NCL=9      # number of clusters
Cdamp=0.95 # reduction of out-of-cluster similarity


aa=0.3; aa2=3e-2; N_iter_G=10; NCL=10; niter_g=5; Cdamp=0.975;
aa2=3e-1;

aa=0.1; aa2=5e-1; N_iter_G=10; NCL=10; niter_g=5; Cdamp=0.85;


## FROM GRID SEARCH
aa=0.4; aa2=3e-1; N_iter_G=10; NCL=10; niter_g=5; Cdamp=0.9; NF = 1;
#_a0.4_t0.3_NF1_CL10_CLd0.9_ie5

aa=0.3; aa2=3e-1; N_iter_G=10; NCL=10; niter_g=3; Cdamp=0.9; NF = 2;
#_a0.3_t0.3_NF2_CL10_CLd0.9_ie3

aa=0.1; aa2=3e-1; N_iter_G=10; NCL=10; niter_g=3; Cdamp=0.9; NF = 3;
#_a0.1_t0.3_NF3_CL10_CLd0.9_ie3
 


SAVE_OUT=0

DO_AUTOGRAD=0 # Use Autograd instead of manual grad
DO_PCA=0 # initialize embedding with PCA

#KMEANS_sklearn=0

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
# FORCE CPU for timing purpose
#device = torch.device("cpu")

iNF=int(NF)

print(device)

cuda


In [2]:
# EVALUATE PERFORMANCE OF DGE
TOL=5
import itertools


bestIt=1; bestWin=8;

all_scores = {}
scores = evaluation_measures(win=5)

MAXW=10

myFM=np.zeros( (N_SEQ,N_iter_G,MAXW) )
myPR=np.zeros( (N_SEQ,N_iter_G,MAXW) )
myRE=np.zeros( (N_SEQ,N_iter_G,MAXW) )

for subid in range(N_SEQ): 
    GT = np.load(path_dir + "GT/seq_" + str(subid+1)+".npy")
    start_time1 = time.time()
    
    #--- LOAD EDUB features
    FILENAME= FPTH + SNME
    FN= SNME + '_seq' + str(subid+1) 
    FN_DGE= SNME + '_seq' + str(subid+1) + '_DGE'
    print(FN)

    #######################################################
    #--- PART 1: NLSS : compute similarity and do nonlocal mean
    FN1 = SAVEDIR + FN + NLSstr + '.npy' 
    GST = "_man"
    
    PCAstr = ""
    FN2 = SAVEDIR + FN + NLSstr + embstr + GST + PCAstr + '.npy'

    #######################################################
    #--- PART 3: DGE LOOP
    
    
    
    for nDGE in range(N_iter_G):
        
        FN3 = SAVEDIR + FN + NLSstr + embstr + '_it' + str(niter_e) + GST + PCAstr + '_it' + str(nDGE+1) + '_DGE.npy'

        if iNF>1:
            FN3 = SAVEDIR + FN + NLSstr + embstr + '_it' + str(niter_e)  + GST +'_NF' + str(iNF) + PCAstr + '_it' + str(nDGE+1) + '_DGE.npy'; 
        

        
        dataiter = np.load(FN3)
        
        if nDGE == bestIt:
            FN4 = SAVEDIR + 'DGE_FINAL_FEATURES/' + FN_DGE + '.npy'
            np.save(FN4,dataiter)
            print(FN4)
        
#        print(dataiter.shape)
        
        for nwindow in range(2,MAXW):
            P, F = mean_aggregation(dataiter, N=nwindow)
            prediction =  boundary_prediction(P,F)
            scores.reset(y_pred=prediction, y_gt=GT, win = TOL)            
            myFM[subid,nDGE,nwindow]=scores.f_measure()
            myPR[subid,nDGE,nwindow]=scores.get_precision()
            myRE[subid,nDGE,nwindow]=scores.get_recall()
            
            

torch.cuda.empty_cache() 


meanFM=myFM.mean(axis=0)
meanPR=myPR.mean(axis=0)
meanRE=myRE.mean(axis=0)

maxFM=meanFM.max(); 
#idM1=meanFM.argmax(axis=0); idM2=meanFM[idM1,:].argmax(axis=1);
idM1,idM2 = np.where(meanFM==meanFM.max())
maxPR=meanPR[idM1,idM2][0]
maxRE=meanRE[idM1,idM2][0]

print(idM1[0],idM2[0])
print(maxFM,maxPR,maxRE)


# PRINT FOR FIXED/ALL WINDOW AND ITERATION
bestIt=1; bestWin=8;
print(bestIt,bestWin)
print(meanFM[bestIt,bestWin],meanPR[bestIt,bestWin],meanRE[bestIt,bestWin])

NameError: name 'evaluation_measures' is not defined

In [63]:
meanFM=myFM.mean(axis=0)
meanPR=myPR.mean(axis=0)
meanRE=myRE.mean(axis=0)

maxFM=meanFM.max(); 
#idM1=meanFM.argmax(axis=0); idM2=meanFM[idM1,:].argmax(axis=1);
idM1,idM2 = np.where(meanFM==meanFM.max())
maxPR=meanPR[idM1,idM2][0]
maxRE=meanRE[idM1,idM2][0]

print(idM1[0],idM2[0])
print(maxFM,maxPR,maxRE)


# PRINT FOR FIXED/ALL WINDOW AND ITERATION
bestIt=1; bestWin=8;
print(bestIt,bestWin)
print(meanFM[bestIt,bestWin],meanPR[bestIt,bestWin],meanRE[bestIt,bestWin])

1 8
0.6975119826831833 0.7214731356059618 0.7027570148456753
1 8
0.6975119826831833 0.7214731356059618 0.7027570148456753


In [ ]:
print(FN + NLSstr + embstr + '_it' + str(niter_e) + GST + PCAstr + '_it' + str(nDGE+1) + '_DGE.npy')
print(FN3)
print(maxFM,maxPR,maxRE)


In [70]:
print('FM')
print(meanFM[:,bestWin])
print('PR')
print(meanPR[:,bestWin])
print('RE')
print(meanRE[:,bestWin])


FM
[0.65529762 0.69751198 0.68165592 0.67126756 0.66236916 0.65233191
 0.64454644 0.64448353 0.63596542 0.6295629 ]
PR
[0.68776772 0.72147314 0.70686499 0.68882751 0.67781854 0.66507126
 0.66372263 0.66268959 0.65997504 0.65501122]
RE
[0.65896458 0.70275701 0.68856242 0.68777862 0.6832521  0.67588988
 0.6692537  0.67036481 0.66251225 0.6572062 ]


In [ ]:
# PRINT FOR ALL SUBJECTS
print(np.transpose([np.transpose(myFM[:,idM1,idM2])[0],np.transpose(myPR[:,idM1,idM2])[0],np.transpose(myRE[:,idM1,idM2])[0]]))

In [7]:
# EVALUATE PERFORMANCE OF NLSS and EMBEDDING

TOL = 5

import itertools
start_time = time.time()

all_scores = {}
scores = evaluation_measures(win=5)

MAXW=10

myFM0=np.zeros( (N_SEQ,MAXW) )
myPR0=np.zeros( (N_SEQ,MAXW) )
myRE0=np.zeros( (N_SEQ,MAXW) )

myFM1=np.zeros( (N_SEQ,MAXW) )
myPR1=np.zeros( (N_SEQ,MAXW) )
myRE1=np.zeros( (N_SEQ,MAXW) )

myFM2=np.zeros( (N_SEQ,MAXW) )
myPR2=np.zeros( (N_SEQ,MAXW) )
myRE2=np.zeros( (N_SEQ,MAXW) )

for subid in range(N_SEQ): 
    GT = np.load(path_dir + "GT/seq_" + str(subid+1)+".npy")
    start_time1 = time.time()
    
    #--- LOAD EDUB features
    FILENAME= FPTH + SNME
    FN= SNME + '_seq' + str(subid+1) 
    print(FN)

    dataiter = np.load(FILENAME + '_seq' + str(subid+1) + '.npy')
    
    for nwindow in range(2,MAXW):
        P, F = mean_aggregation(dataiter, N=nwindow)
        prediction =  boundary_prediction(P,F)
        scores.reset(y_pred=prediction, y_gt=GT, win = TOL)            
        myFM0[subid,nwindow]=scores.f_measure()
        myPR0[subid,nwindow]=scores.get_precision()
        myRE0[subid,nwindow]=scores.get_recall()
        
    #######################################################
    #--- PART 1: NLSS : compute similarity and do nonlocal mean
    FN1 = SAVEDIR + FN + NLSstr + '.npy' 
    GST = "_man"
    
    dataiter = np.load(FN1)        
    for nwindow in range(2,MAXW):
        P, F = mean_aggregation(dataiter, N=nwindow)
        prediction =  boundary_prediction(P,F)
        scores.reset(y_pred=prediction, y_gt=GT, win = TOL)            
        myFM1[subid,nwindow]=scores.f_measure()
        myPR1[subid,nwindow]=scores.get_precision()
        myRE1[subid,nwindow]=scores.get_recall()
        
        if nwindow==MAXW-1:
            FN= SNME + '_seq' + str(subid+1)
            np.save(SAVEDIR + FN + '_NLSS_P' + '.npy' ,P)
            np.save(SAVEDIR + FN + '_NLSS_F' + '.npy' ,F)
            
    
    PCAstr = ""
    FN2 = SAVEDIR + FN + NLSstr + embstr + '_it' + str(niter_e) + GST + PCAstr + '.npy'
    dataiter = np.load(FN2)        
    for nwindow in range(2,MAXW):
        P, F = mean_aggregation(dataiter, N=nwindow)
        prediction =  boundary_prediction(P,F)
        scores.reset(y_pred=prediction, y_gt=GT, win = TOL)            
        myFM2[subid,nwindow]=scores.f_measure()
        myPR2[subid,nwindow]=scores.get_precision()
        myRE2[subid,nwindow]=scores.get_recall()
            
            

print("--- %s seconds --------" % (time.time() - start_time))
torch.cuda.empty_cache() 

meanFM0=myFM0.mean(axis=0)
meanPR0=myPR0.mean(axis=0)
meanRE0=myRE0.mean(axis=0)
meanFM1=myFM1.mean(axis=0)
meanPR1=myPR1.mean(axis=0)
meanRE1=myRE1.mean(axis=0)
meanFM2=myFM2.mean(axis=0)
meanPR2=myPR2.mean(axis=0)
meanRE2=myRE2.mean(axis=0)

EDUB-Seg_seq1
EDUB-Seg_seq2
EDUB-Seg_seq3
EDUB-Seg_seq4
EDUB-Seg_seq5
EDUB-Seg_seq6
EDUB-Seg_seq7
EDUB-Seg_seq8
EDUB-Seg_seq9
EDUB-Seg_seq10
EDUB-Seg_seq11
EDUB-Seg_seq12
EDUB-Seg_seq13
EDUB-Seg_seq14
EDUB-Seg_seq15
EDUB-Seg_seq16
EDUB-Seg_seq17
EDUB-Seg_seq18
EDUB-Seg_seq19
EDUB-Seg_seq20
--- 62.58055138587952 seconds --------


In [8]:
stdFM0=myFM0.std(axis=0)
stdPR0=myPR0.std(axis=0)
stdRE0=myRE0.std(axis=0)
stdFM1=myFM1.std(axis=0)
stdPR1=myPR1.std(axis=0)
stdRE1=myRE1.std(axis=0)
stdFM2=myFM2.std(axis=0)
stdPR2=myPR2.std(axis=0)
stdRE2=myRE2.std(axis=0)

print('WIN='+str(MAXW))
print('RAW  ' + str(stdFM0[MAXW-1]) + ' ' + str(stdPR0[MAXW-1]) + ' ' + str(stdRE0[MAXW-1]))
print('NLSS ' + str(stdFM1[MAXW-1]) + ' ' + str(stdPR1[MAXW-1]) + ' ' + str(stdRE1[MAXW-1]))
print('EMBD ' + str(stdFM2[MAXW-1]) + ' ' + str(stdPR2[MAXW-1]) + ' ' + str(stdRE2[MAXW-1]))

print('WIN='+str(bestWin))
print('RAW  ' + str(stdFM0[bestWin-1]) + ' ' + str(stdPR0[bestWin-1]) + ' ' + str(stdRE0[bestWin-1]))
print('NLSS ' + str(stdFM1[bestWin-1]) + ' ' + str(stdPR1[bestWin-1]) + ' ' + str(stdRE1[bestWin-1]))
print('EMBD ' + str(stdFM2[bestWin-1]) + ' ' + str(stdPR2[bestWin-1]) + ' ' + str(stdRE2[bestWin-1]))

WIN=10
RAW  0.09457152725419418 0.16250524462950044 0.139946534740702
NLSS 0.07874094812657187 0.15203729358599155 0.1374194058821018
EMBD 0.0796785615876045 0.12095165034759334 0.12618683544110407
WIN=8
RAW  0.09120202738456545 0.16791995567356452 0.135067592627256
NLSS 0.06643914458696144 0.14461087352269153 0.13125616865679143
EMBD 0.07396582648964332 0.1261665727003643 0.11958366185245421


In [15]:
# EVALUATE PERFORMANCE OF RAW FEATURES

TOL = 5

bestIt=1; bestWin=8; MAXW=10;

import itertools
start_time = time.time()

all_scores = {}
scores = evaluation_measures(win=5)

MAXW=10

myFM3=np.zeros( (N_SEQ,MAXW) )
myPR3=np.zeros( (N_SEQ,MAXW) )
myRE3=np.zeros( (N_SEQ,MAXW) )

NN=np.zeros(N_SEQ)

for subid in range(N_SEQ): 
    GT = np.load(path_dir + "GT/seq_" + str(subid+1)+".npy")
    start_time1 = time.time()
    
    #--- LOAD EDUB features
    FEATDIR = ''
    FILENAME= FPTH + SNME
    FN= SNME + '_seq' + str(subid+1) 
    dataiter = np.load(FEATDIR + FN + '.npy')
    
    #print(np.min(dataiter),np.max(dataiter))
    
    tmp=dataiter.shape
    #print(tmp[0])
    #dataiter=2*(dataiter-1/2)
    NN[subid]=tmp[0]
    
    for nwindow in range(2,MAXW):
        P, F = mean_aggregation(dataiter, N=nwindow)
        prediction =  boundary_prediction(P,F)
        scores.reset(y_pred=prediction, y_gt=GT, win = TOL)            
        myFM3[subid,nwindow]=scores.f_measure()
        myPR3[subid,nwindow]=scores.get_precision()
        myRE3[subid,nwindow]=scores.get_recall()
                
            

print("--- %s seconds --------" % (time.time() - start_time))
torch.cuda.empty_cache() 

print(' ')
print(NN.min())
print(NN.mean())
print(NN.max())




--- 32.47061204910278 seconds --------
 
269.0
935.95
1752.0
